# Data Preprocessing: Trip Duration

Generate code & functions such that conducts data preprocessing(includes feature engineering & data cleaning).

In [1]:
%%time

# Import Standard Libraries
import os
import sys
import warnings
# warnings.filterwarnings("ignore")

# Import Data Handling Libraries
import pandas as pd
import numpy as np
np.random.seed(42)

# Import Date-Time Handling Libraries
from datetime import timedelta
import datetime as dt

# Import Geodetic Libraries
import pyproj
from pyproj import Geod

# Import Data Visualization Libraries
import matplotlib
matplotlib.rcParams["font.size"] = 12
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [12, 12]  # Set default figure size
import seaborn as sns

# Import Machine Learning Libraries
from sklearn.decomposition import PCA  # Principal Component Analysis

# Set random seed for reproducibility in scikit-learn
from sklearn.utils import check_random_state
rng = check_random_state(42)

# Import Utilities
import gc
from tqdm import tqdm
import joblib
import json

# Import Custom Modules
from data_loader import *  # Custom data loading functions

CPU times: total: 4.56 s
Wall time: 5.8 s


In [22]:
%%time

# Load the dataset
df_train = load_data("train")
df_train.head()

Detected environment: sys
Loading train.csv from: E:\Term 2\[CEGE0042] Spatial-Temporal Data Analysis & Mining\Assignment\Repository\data\train.csv
CPU times: total: 4.19 s
Wall time: 4.2 s


,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id0458976,2,2016-06-29 18:21:02,2016-06-29 18:39:55,1,-73.862762,40.768822,-73.891701,40.746689,N,1133
1,id0434613,2,2016-04-25 13:03:26,2016-04-25 13:18:13,1,-73.958038,40.783237,-73.975510,40.760853,N,887
2,id3809234,2,2016-05-07 12:36:09,2016-05-07 12:47:35,1,-73.969460,40.785519,-73.989243,40.771748,N,686
3,id1203705,1,2016-05-14 18:44:17,2016-05-14 18:57:55,1,-73.981743,40.736549,-73.998352,40.726440,N,818
4,id1896645,2,2016-04-10 22:51:25,2016-04-10 23:07:16,1,-73.977913,40.752609,-73.975647,40.733139,N,951


In [3]:
%%time

# Load the test dataset
df_test = load_data("test")
df_test.head()

Detected environment: sys
Loading test.csv from: E:\Term 2\[CEGE0042] Spatial-Temporal Data Analysis & Mining\Assignment\Repository\data\test.csv
CPU times: total: 1.23 s
Wall time: 1.23 s


,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2793718,2,2016-06-08 07:36:19,2016-06-08 07:53:39,1,-73.985611,40.735943,-73.980331,40.760468,N,1040
1,id3485529,2,2016-04-03 12:58:11,2016-04-03 13:11:58,1,-73.978394,40.764351,-73.991623,40.749859,N,827
2,id1816614,2,2016-06-05 02:49:13,2016-06-05 02:59:27,5,-73.989059,40.744389,-73.973381,40.748692,N,614
3,id1050851,2,2016-05-05 17:18:27,2016-05-05 17:32:54,2,-73.990326,40.731136,-73.991264,40.748917,N,867
4,id0140657,1,2016-05-12 17:43:38,2016-05-12 19:06:25,4,-73.789497,40.646675,-73.987137,40.759232,N,4967


In [4]:
%%time

# Delete cols that leads to data leakage
del df_train["dropoff_datetime"]
gc.collect()

CPU times: total: 109 ms
Wall time: 111 ms


0

## Feature Extraction

### PCA in Longitudes & Latitudes

In [5]:
%%time

def apply_pca_to_coords(train, test, random_seed=42):
    """
    Applies PCA transformation to pickup and dropoff coordinates for train and test datasets.

    The PCA is fitted **only on the training data** to prevent data leakage.

    Parameters:
        train (pd.DataFrame): The training dataset.
        test (pd.DataFrame): The testing dataset.
        random_seed (int): Random seed for reproducibility.

    Returns:
        None: Modifies train and test DataFrames in place.
    """

    # Fit PCA only on training data
    coords_train = np.vstack((
        train[["pickup_latitude", "pickup_longitude"]].values,
        train[["dropoff_latitude", "dropoff_longitude"]].values
    ))

    pca = PCA(whiten=True, random_state=random_seed).fit(coords_train)

    # Apply transformation to train dataset
    train_coords_pickup = train[["pickup_latitude", "pickup_longitude"]].values
    train_coords_dropoff = train[["dropoff_latitude", "dropoff_longitude"]].values
    train.loc[:, "pickup_pca0"] = pca.transform(train_coords_pickup)[:, 0]
    train.loc[:, "pickup_pca1"] = pca.transform(train_coords_pickup)[:, 1]
    train.loc[:, "dropoff_pca0"] = pca.transform(train_coords_dropoff)[:, 0]
    train.loc[:, "dropoff_pca1"] = pca.transform(train_coords_dropoff)[:, 1]

    # Apply the same transformation to test dataset to avoid data leakage
    test_coords_pickup = test[["pickup_latitude", "pickup_longitude"]].values
    test_coords_dropoff = test[["dropoff_latitude", "dropoff_longitude"]].values
    test.loc[:, "pickup_pca0"] = pca.transform(test_coords_pickup)[:, 0]
    test.loc[:, "pickup_pca1"] = pca.transform(test_coords_pickup)[:, 1]
    test.loc[:, "dropoff_pca0"] = pca.transform(test_coords_dropoff)[:, 0]
    test.loc[:, "dropoff_pca1"] = pca.transform(test_coords_dropoff)[:, 1]

# Example usage:
apply_pca_to_coords(df_train, df_test, random_seed=42)

CPU times: total: 2.17 s
Wall time: 533 ms


### Distance

In [6]:
%%time

# Define WGS84 ellipsoid
geod = Geod(ellps="WGS84")

# Compute great-circle distance in kilometers
df_train["euclidean_distance"] = df_train.apply(
    lambda row: geod.inv(row["pickup_longitude"], row["pickup_latitude"],
                         row["dropoff_longitude"], row["dropoff_latitude"])[2] / 1000, axis=1
)

# Compute great-circle distance in kilometers
df_test["euclidean_distance"] = df_test.apply(
    lambda row: geod.inv(row["pickup_longitude"], row["pickup_latitude"],
                         row["dropoff_longitude"], row["dropoff_latitude"])[2] / 1000, axis=1
)

CPU times: total: 38.9 s
Wall time: 38.2 s


### Datetime Feature

In [7]:
%%time

def generate_datetime_features(df):
    """
    Generate detailed date-time features for pickups and modify the DataFrame in place.

    Parameters:
        df (pd.DataFrame): The DataFrame containing the datetime column.
    
    Returns:
        None (Modifies df in place)
    """
    # Convert to datetime format
    pickup_times = pd.to_datetime(df["pickup_datetime"])

    # Extract relevant time features
    df["pickup_hour_of_day"] = (pickup_times.dt.hour * 60.0 + pickup_times.dt.minute) / 60.0

    df["day_of_week"] = pickup_times.dt.weekday
    df["hour_of_week"] = df["day_of_week"] * 24.0 + df["pickup_hour_of_day"]

    df["month_of_year"] = pickup_times.dt.month
    df["day_of_year"] = pickup_times.dt.dayofyear
    df["week_of_year"] = pickup_times.dt.isocalendar().week
    df["hour_of_year"] = df["day_of_year"] * 24.0 + df["pickup_hour_of_day"]

generate_datetime_features(df_train)
generate_datetime_features(df_test)

CPU times: total: 1.83 s
Wall time: 1.61 s


### NYC Weather

In [8]:
%%time

def merge_weather_data(df):
    """
    Merges weather data with a given dataframe (train or test) based on the pickup date.

    Parameters:
        df (pd.DataFrame): The train or test dataframe containing 'pickup_datetime'.
    weather_data (pd.DataFrame): The weather data dataframe containing 'date'.

    Returns:
        None (Modifies df in place)
    """

    # Load nyc weather data to enrich information
    weather_data = pd.read_csv(os.path.join("utils",
                                            "weather_data_nyc_centralpark_2016.csv"))
    weather_data["date"] = pd.to_datetime(weather_data["date"], format="%d-%m-%Y")

    # Ensure datetime consistency in datetime format
    weather_data["date"] = weather_data["date"].dt.date
    df["pickup_date"] = pd.to_datetime(df["pickup_datetime"]).dt.date

    # Merge datasets on the date
    df = df.merge(weather_data, left_on="pickup_date", right_on="date", how="left")

    # Drop redundant date columns
    df.drop(columns=["pickup_date"], inplace=True)

# Apply function to train and test datasets
merge_weather_data(df_train)
merge_weather_data(df_test)

CPU times: total: 2.3 s
Wall time: 2.31 s


## Data Cleaning

### Location Outlier

In [9]:
%%time

def filter_by_nyc_boundary(df, geojson_path):
    """
    Filters pickup and dropoff locations to keep only those within the New York City boundary.

    Parameters:
        df (pd.DataFrame): The DataFrame containing pickup and dropoff coordinates.
        geojson_path (str): Path to the GeoJSON file defining NYC boundaries.

    Returns:
        pd.DataFrame: Filtered DataFrame with locations inside the NYC bounding box.
    """
    # Load the GeoJSON file
    with open(geojson_path, "r") as f:
        geojson_data = json.load(f)

    # Extract NYC boundary coordinates where NAME is "New York"
    nyc_coords = []
    for feature in geojson_data["features"]:
        if feature["properties"].get("NAME") == "New York":
            for polygon in feature["geometry"]["coordinates"]:  # Loop through MultiPolygon
                for ring in polygon:  # Each polygon has a ring of coordinates
                    nyc_coords.extend(ring)

    # Compute NYC bounding box (min/max latitudes & longitudes)
    min_long = min(lon for lon, lat in nyc_coords)
    max_long = max(lon for lon, lat in nyc_coords)
    min_lat = min(lat for lon, lat in nyc_coords)
    max_lat = max(lat for lon, lat in nyc_coords)

    # Count records before filtering
    initial_count = len(df)

    # Filter data based on bounding box
    mask = (
        (df["pickup_longitude"].between(min_long, max_long))
        & (df["pickup_latitude"].between(min_lat, max_lat))
        & (df["dropoff_longitude"].between(min_long, max_long))
        & (df["dropoff_latitude"].between(min_lat, max_lat))
    )

    filtered_df = df[mask]

    # Count records after filtering
    final_count = len(filtered_df)
    dropped_count = initial_count - final_count

    print(f"Records before filtering: {initial_count}")
    print(f"Records after filtering: {final_count}")
    print(f"Records dropped: {dropped_count}\n")

    return filtered_df


# Apply function to df_train and df_test
df_train = filter_by_nyc_boundary(df_train, "utils/gz_2010_us_040_00_5m.json")
df_test = filter_by_nyc_boundary(df_test, "utils/gz_2010_us_040_00_5m.json")

Records before filtering: 1166915
Records after filtering: 1166864
Records dropped: 51

Records before filtering: 291729
Records after filtering: 291713
Records dropped: 16

CPU times: total: 703 ms
Wall time: 701 ms


### Duration Outlier

In [10]:
%%time

def filter_by_duration_range(df, lower_lim, upper_lim):
    """
    Filters trips based on duration range.

    Parameters:
        df (pd.DataFrame): The DataFrame containing trip durations.
        lower_lim (float): The minimum allowable duration.
        upper_lim (float): The maximum allowable duration.

    Returns:
        pd.DataFrame: Filtered DataFrame with durations within the specified range.
    """
    initial_count = len(df)
    
    # Apply duration filter
    filtered_df = df[df["trip_duration"].between(lower_lim, upper_lim)]

    final_count = len(filtered_df)
    dropped_count = initial_count - final_count

    print(f"Records before filtering: {initial_count}")
    print(f"Records after filtering: {final_count}")
    print(f"Records dropped due to duration limits: {dropped_count}")

    return filtered_df

# Apply function to df_train
df_train = filter_by_duration_range(df_train, 5, 36000)

Records before filtering: 1166864
Records after filtering: 1164588
Records dropped due to duration limits: 2276
CPU times: total: 250 ms
Wall time: 285 ms


### Spatial & Temporal Aggregation

- Spatial Aggregation of Trip Counts

In [22]:
%%time

def bin_coordinates(df, precision=2):
    """Bins latitude and longitude to a specified precision."""
    df.loc[:, "pickup_lat_bin"] = np.round(df["pickup_latitude"], precision)
    df.loc[:, "pickup_long_bin"] = np.round(df["pickup_longitude"], precision)
    df.loc[:, "dropoff_lat_bin"] = np.round(df["dropoff_latitude"], precision)
    df.loc[:, "dropoff_long_bin"] = np.round(df["dropoff_longitude"], precision)

bin_coordinates(df_train)
bin_coordinates(df_test)

CPU times: total: 78.1 ms
Wall time: 61.1 ms


In [23]:
%%time

def compute_spatial_aggregations(df, min_trips=100):
    """Computes trip counts for different spatial aggregations."""
    groupings = [
        ["pickup_lat_bin", "pickup_long_bin", "dropoff_lat_bin", "dropoff_long_bin"],
        ["pickup_lat_bin", "pickup_long_bin"],
        ["dropoff_lat_bin", "dropoff_long_bin"]
    ]
    
    for groupby_cols in groupings:
        col_name = "cnt_coords_bin_" + "".join(set([col[0] for col in groupby_cols]))
        
        # Compute trip counts and store in a dictionary for fast lookup
        counts = df.groupby(groupby_cols).size().to_dict()
        
        # Apply counts to create a new column in the dataframe
        df[col_name] = df[groupby_cols].apply(lambda row: counts.get(tuple(row), 0), axis=1)
        
        # Apply filtering based on min_trips
        df[col_name] = df[col_name].where(df[col_name] >= min_trips, 0)

compute_spatial_aggregations(df_train)
compute_spatial_aggregations(df_test)

KeyboardInterrupt: 

- Temporal Aggregation of Trip Counts

In [77]:
%%time

# Load the dataset
df_train = load_data("train")
df_test = load_data("test")

bin_coordinates(df_train)
bin_coordinates(df_test)

Detected environment: sys
Loading train.csv from: E:\Term 2\[CEGE0042] Spatial-Temporal Data Analysis & Mining\Assignment\Repository\data\train.csv
Detected environment: sys
Loading test.csv from: E:\Term 2\[CEGE0042] Spatial-Temporal Data Analysis & Mining\Assignment\Repository\data\test.csv
CPU times: total: 5.77 s
Wall time: 5.78 s


In [78]:
%%time

def compute_temporal_aggregation(df, df_ref, n_jobs=-1):
    """
    Compute temporal aggregated features for taxi trips
    
    Args:
        df: DataFrame to compute features for (must contain pickup/dropoff bins and timestamps)
        df_ref: Reference DataFrame used for aggregation calculations
    
    Adds three new columns:
        prev_1h_trip_count: Total trips in previous 1 hour window
        mean_prev_3h_pickups_trip_count: Average hourly pickups in matching spatial bin (T-4h to T-1h)
        mean_prev_3h_dropoffs_trip_count: Average hourly dropoffs in matching spatial bin (T-4h to T-1h)
    """
    # Ensure pickup_datetime is in datetime format
    df["pickup_datetime"] = pd.to_datetime(df["pickup_datetime"])
    df_ref["pickup_datetime"] = pd.to_datetime(df_ref["pickup_datetime"])
    
    # Initialize progress bar
    tqdm.pandas(desc="Processing trips")
    
    # Create temporary columns for time calculations
    df_ref = df_ref.copy()
    df_ref["pickup_hour"] = df_ref["pickup_datetime"].dt.floor("H")
    
    # Add new columns with default values
    df["prev_1h_trip_count"] = 0
    df["mean_prev_3h_pickups_trip_count"] = 0.0
    df["mean_prev_3h_dropoffs_trip_count"] = 0.0
    
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        # Get current trip attributes
        current_time = row["pickup_datetime"]
        current_hour = current_time.floor("H")
        pl_bin = row["pickup_lat_bin"]
        plon_bin = row["pickup_long_bin"]
        dl_bin = row["dropoff_lat_bin"]
        dlon_bin = row["dropoff_long_bin"]
        
        # Calculate 1-hour window
        t1_start = current_hour - pd.Timedelta(hours=1)
        t1_end = current_hour
        
        # Calculate 3-hour average window (T-4h to T-1h)
        t3_start = current_hour - pd.Timedelta(hours=4)
        t3_end = current_hour - pd.Timedelta(hours=1)
        
        # Get reference data subsets
        ref_1h = df_ref[
            (df_ref["pickup_hour"] >= t1_start) & 
            (df_ref["pickup_hour"] < t1_end)
        ]
        
        ref_3h = df_ref[
            (df_ref["pickup_hour"] >= t3_start) & 
            (df_ref["pickup_hour"] < t3_end)
        ]
        
        # Calculate 1-hour total count
        df.at[idx, "prev_1h_trip_count"] = len(ref_1h)
        
        # Calculate 3-hour spatial averages
        pickup_count = len(ref_3h[
            (ref_3h["pickup_lat_bin"] == pl_bin) &
            (ref_3h["pickup_long_bin"] == plon_bin)
        ])
        
        dropoff_count = len(ref_3h[
            (ref_3h["dropoff_lat_bin"] == dl_bin) &
            (ref_3h["dropoff_long_bin"] == dlon_bin)
        ])
        
        df.at[idx, "mean_prev_3h_pickups_trip_count"] = pickup_count / 3
        df.at[idx, "mean_prev_3h_dropoffs_trip_count"] = dropoff_count / 3
        
    return df

# For training data:
df_train = compute_temporal_aggregation(df_train, df_train)

# For test data:
combined_ref = pd.concat([df_train, df_test], ignore_index=True)
df_test = compute_temporal_aggregation(df_test, combined_ref)

  0%|▎                                                                        | 5813/1166915 [02:49<9:23:45, 34.33it/s]


KeyboardInterrupt: 

In [ ]:
%%time

import pandas as pd
from tqdm import tqdm
from joblib import Parallel, delayed

def process_chunk(chunk, df_ref):
    """
    Process a chunk of the DataFrame to compute spatial-temporal features.
    
    Args:
        chunk: A subset of the main DataFrame.
        df_ref: Reference DataFrame for aggregation calculations.
    
    Returns:
        DataFrame with computed features for the chunk.
    """
    # Ensure pickup_datetime is in datetime format
    chunk["pickup_datetime"] = pd.to_datetime(chunk["pickup_datetime"])
    df_ref["pickup_datetime"] = pd.to_datetime(df_ref["pickup_datetime"])
    
    # Create temporary columns for time calculations
    df_ref = df_ref.copy()
    df_ref["pickup_hour"] = df_ref["pickup_datetime"].dt.floor("H")
    
    # Add new columns with default values
    chunk["prev_1h_trip_count"] = 0
    chunk["mean_prev_3h_pickups_trip_count"] = 0.0
    chunk["mean_prev_3h_dropoffs_trip_count"] = 0.0
    
    for idx, row in chunk.iterrows():
        # Get current trip attributes
        current_time = row["pickup_datetime"]
        current_hour = current_time.floor("H")
        pl_bin = row["pickup_lat_bin"]
        plon_bin = row["pickup_long_bin"]
        dl_bin = row["dropoff_lat_bin"]
        dlon_bin = row["dropoff_long_bin"]
        
        # Calculate 1-hour window
        t1_start = current_hour - pd.Timedelta(hours=1)
        t1_end = current_hour
        
        # Calculate 3-hour average window (T-4h to T-1h)
        t3_start = current_hour - pd.Timedelta(hours=4)
        t3_end = current_hour - pd.Timedelta(hours=1)
        
        # Get reference data subsets
        ref_1h = df_ref[
            (df_ref["pickup_hour"] >= t1_start) & 
            (df_ref["pickup_hour"] < t1_end)
        ]
        
        ref_3h = df_ref[
            (df_ref["pickup_hour"] >= t3_start) & 
            (df_ref["pickup_hour"] < t3_end)
        ]
        
        # Calculate 1-hour total count
        chunk.at[idx, "prev_1h_trip_count"] = len(ref_1h)
        
        # Calculate 3-hour spatial averages
        pickup_count = len(ref_3h[
            (ref_3h["pickup_lat_bin"] == pl_bin) &
            (ref_3h["pickup_long_bin"] == plon_bin)
        ])
        
        dropoff_count = len(ref_3h[
            (ref_3h["dropoff_lat_bin"] == dl_bin) &
            (ref_3h["dropoff_long_bin"] == dlon_bin)
        ])
        
        chunk.at[idx, "mean_prev_3h_pickups_trip_count"] = pickup_count / 3
        chunk.at[idx, "mean_prev_3h_dropoffs_trip_count"] = dropoff_count / 3
    
    return chunk

def compute_spatial_aggregation_parallel(df, df_ref, n_jobs=-1):
    """
    Compute spatial-temporal aggregated features for taxi trips in parallel.
    
    Args:
        df: DataFrame to compute features for (must contain pickup/dropoff bins and timestamps).
        df_ref: Reference DataFrame used for aggregation calculations.
        n_jobs: Number of parallel jobs to run. Default is -1 (use all available cores).
    
    Returns:
        DataFrame with computed features.
    """
    # Split the DataFrame into chunks for parallel processing
    chunks = [df.iloc[i:i + 1000] for i in range(0, len(df), 1000)]
    
    # Process chunks in parallel
    results = Parallel(n_jobs=n_jobs)(
        delayed(process_chunk)(chunk, df_ref) for chunk in tqdm(chunks, desc="Processing chunks")
    )
    
    # Combine results into a single DataFrame
    return pd.concat(results, ignore_index=True)

# For training data:
df_train = compute_temporal_aggregation(df_train, df_train)

# For test data:
combined_ref = pd.concat([df_train, df_test], ignore_index=True)
df_test = compute_temporal_aggregation(df_test, combined_ref)

In [73]:
import pandas as pd
import random

def check_trip_counts(df, k=3):
    # 随机选择k=3条记录
    selected_records = df.sample(n=k, random_state=42)
    
    # 用于保存检查结果的列表
    results = []
    
    for idx, record in selected_records.iterrows():
        pickup_datetime = record['pickup_datetime']
        pickup_lat_bin = record['pickup_lat_bin']
        pickup_long_bin = record['pickup_long_bin']
        dropoff_lat_bin = record['dropoff_lat_bin']
        dropoff_long_bin = record['dropoff_long_bin']
        id_value = record['id']  # 获取id
        
        # 获取前1小时的时间窗口，向下取整到整小时
        time_minus_1h_start = pd.to_datetime(pickup_datetime.replace(minute=0, second=0, microsecond=0)) - pd.Timedelta(hours=1)
        time_minus_1h_end = pd.to_datetime(pickup_datetime.replace(minute=0, second=0, microsecond=0))
        
        # 获取前1小时范围内的记录并计算计数
        count_Tminus1h = len(df[(df['pickup_datetime'] >= time_minus_1h_start) & 
                                (df['pickup_datetime'] < time_minus_1h_end)])
        
        # 检查前4小时到前1小时内，符合pickup条件的记录 (T-4h to T-1h)
        time_minus_4h_start = pd.to_datetime(pickup_datetime.replace(minute=0, second=0, microsecond=0)) - pd.Timedelta(hours=4)
        
        # 获取前4小时到前1小时的记录并计算pickup条件下的计数
        pickup_condition = df[(df['pickup_datetime'] >= time_minus_4h_start) & 
                              (df['pickup_datetime'] < time_minus_1h_end) & 
                              (df['pickup_lat_bin'] == pickup_lat_bin) & 
                              (df['pickup_long_bin'] == pickup_long_bin)]
        count_pickup_Tminus4h_Tminus1h = len(pickup_condition)
        
        # 检查前4小时到前1小时内，符合dropoff条件的记录 (T-4h to T-1h)
        dropoff_condition = df[(df['pickup_datetime'] >= time_minus_4h_start) & 
                               (df['pickup_datetime'] < time_minus_1h_end) & 
                               (df['dropoff_lat_bin'] == dropoff_lat_bin) & 
                               (df['dropoff_long_bin'] == dropoff_long_bin)]
        count_dropoff_Tminus4h_Tminus1h = len(dropoff_condition)
        
        # 存储结果
        results.append({
            "id": id_value,  # 添加id
            "pickup_datetime": pickup_datetime,
            "mean_cnt_Tminus1h": count_Tminus1h,
            "mean_cnt_pickup_trips_Tminus4h_Tminus1h": count_pickup_Tminus4h_Tminus1h,
            "mean_cnt_dropoff_trips_Tminus4h_Tminus1h": count_dropoff_Tminus4h_Tminus1h
        })
    
    # 转换为DataFrame返回
    result_df = pd.DataFrame(results)
    
    # 检查每个元素和df中的记录是否相等
    for idx, elem in result_df.iterrows():
        # 根据id获取df中的记录
        row = df[df['id'] == elem['id']].iloc[0]  # 获取df中id对应的记录
        
        # 比较results中的每个key与df中对应的列值是否相等
        for key in elem.index:
            if key != 'id':  # 排除id字段
                if elem[key] == row[key]:
                    print(f"于 {key} 字段，相等： {elem[key]} == {row[key]}")
                else:
                    print(f"于 {key} 字段，不相等： {elem[key]} != {row[key]}")
    
    return result_df

# 使用示例
result_df = check_trip_counts(df_train, k=3)
print(result_df)

于 pickup_datetime 字段，相等： 2016-03-10 15:17:28 == 2016-03-10 15:17:28
于 mean_cnt_Tminus1h 字段，不相等： 322 != 318.0
于 mean_cnt_pickup_trips_Tminus4h_Tminus1h 字段，不相等： 108 != 34.75
于 mean_cnt_dropoff_trips_Tminus4h_Tminus1h 字段，不相等： 58 != 17.75
于 pickup_datetime 字段，相等： 2016-02-20 08:58:35 == 2016-02-20 08:58:35
于 mean_cnt_Tminus1h 字段，不相等： 115 != 152.0
于 mean_cnt_pickup_trips_Tminus4h_Tminus1h 字段，不相等： 12 != 7.5
于 mean_cnt_dropoff_trips_Tminus4h_Tminus1h 字段，不相等： 10 != 9.5
于 pickup_datetime 字段，相等： 2016-05-10 15:08:15 == 2016-05-10 15:08:15
于 mean_cnt_Tminus1h 字段，不相等： 310 != 314.0
于 mean_cnt_pickup_trips_Tminus4h_Tminus1h 字段，不相等： 83 != 29.25
于 mean_cnt_dropoff_trips_Tminus4h_Tminus1h 字段，不相等： 13 != 6.0
          id     pickup_datetime  mean_cnt_Tminus1h  \
0  id3175894 2016-03-10 15:17:28                322   
1  id2218059 2016-02-20 08:58:35                115   
2  id3614707 2016-05-10 15:08:15                310   

   mean_cnt_pickup_trips_Tminus4h_Tminus1h  \
0                                   

### Drop Redundant Columns

In [36]:
df_train.columns

Index(['pickup_timestamp', 'id', 'vendor_id', 'pickup_datetime',
       'dropoff_datetime', 'passenger_count', 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'store_and_fwd_flag', 'trip_duration', 'cnt_trips_last_1h'],
      dtype='object')

In [15]:
df_test.columns

Index(['id', 'vendor_id', 'pickup_datetime', 'dropoff_datetime',
       'passenger_count', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'store_and_fwd_flag',
       'trip_duration', 'pickup_pca0', 'pickup_pca1', 'dropoff_pca0',
       'dropoff_pca1', 'euclidean_distance', 'pickup_hour_of_day',
       'day_of_week', 'hour_of_week', 'month_of_year', 'day_of_year',
       'week_of_year', 'hour_of_year', 'pickup_date', 'pickup_lat_bin',
       'pickup_long_bin', 'dropoff_lat_bin', 'dropoff_long_bin',
       'cnt_coords_bin_pd', 'cnt_coords_bin_p', 'cnt_coords_bin_d'],
      dtype='object')

### Save Data in parquet

In [ ]:
%%time

# Ensure the 'prep' directory exists
os.makedirs(os.path.join("data", "prep"), exist_ok=True)

# Save to Parquet format
df_train.to_parquet("prep/df_train.parquet", index=False)
df_test.to_parquet("prep/df_test.parquet", index=False)

print("df_train and df_test saved to 'prep' directory as Parquet files.")

---